In [53]:
# Input a number - e.g. 150, 
# find list of concepts at different levels (where the input number locates in) from 
# each concept scheme in the whole triple store through looping, 
# and return the list of concepts from each concept scheme

install.packages("SPARQL")
library(SPARQL)

Installing package into ‘/nethome/max/.R-libs’
(as ‘lib’ is unspecified)



In [54]:
# set up end point and prefix
endpoint = "http://virtuoso.nkn.uidaho.edu:8890/sparql/"
  
# attach SPARQL querry prefix. Note: the graph for our study should be updated
sparql_prefix = " 
prefix dc: <http://purl.org/dc/elements/1.1/> 
prefix dcterms: <http://purl.org/dc/terms/> 
prefix foaf: <http://xmlns.com/foaf/0.1/> 
prefix geo: <http://www.opengis.net/ont/geosparql#> 
prefix gts: <http://resource.geosciml.org/ontology/timescale/gts#> 
prefix isc: <http://resource.geosciml.org/classifier/ics/ischart/> 
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rank: <http://resource.geosciml.org/ontology/timescale/rank/> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix samfl: <http://def.seegrid.csiro.au/ontology/om/sam-lite#> 
prefix sf: <http://www.opengis.net/ont/sf#> 
prefix skos: <http://www.w3.org/2004/02/skos/core#> 
prefix sosa: <http://www.w3.org/ns/sosa/> 
prefix thors: <http://resource.geosciml.org/ontology/timescale/thors#> 
prefix time: <http://www.w3.org/2006/time#> 
prefix ts: <http://resource.geosciml.org/vocabulary/timescale/> 
prefix vann: <http://purl.org/vocab/vann/> 
prefix void: <http://rdfs.org/ns/void#> 
prefix xkos: <http://rdf-vocabulary.ddialliance.org/xkos#> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 
prefix tsna: <http://deeptimekb.org/tsna#> 
prefix tssc: <http://deeptimekb.org/tssc#> 
prefix tsnc: <http://deeptimekb.org/tsnc#> 
prefix tsnz: <http://deeptimekb.org/tsnz#> 
prefix tswe: <http://deeptimekb.org/tswe#> 
prefix tsbr: <http://deeptimekb.org/tsbr#> 
prefix tsba: <http://deeptimekb.org/tsba#> 
prefix tsjp: <http://deeptimekb.org/tsjp#> 
prefix tsau: <http://deeptimekb.org/tsau#> 
        "

In [55]:
# Get a list of vocabulary scheme URIs

q0 = paste0(
              sparql_prefix, 
                  '
                   SELECT   DISTINCT  ?schURI  str(?lbl) AS ?schemeName
                   WHERE
                   {
                     GRAPH <http://deeptimekb.org/iscallnew> 
                     {
                        ?schURI a skos:ConceptScheme ;
                             rdfs:label ?lbl .
                     }      
                   }
                  '
              )

# run the query  
schemeList = SPARQL(endpoint, q0)$results
  
schListLength = length(schemeList$schURI)

In [56]:
#this query will take about 10 secs

for (k in 1:schListLength) {

cat("\n\n","========")
print ('Input age boundary number: 150')
print(schemeList$schURI[k])
print(schemeList$schemeName[k]) 
    
# This query will find the Age-level concept 
q0 = paste0( sparql_prefix, '
SELECT DISTINCT ?tconcept  ?lbl
WHERE
{
  GRAPH <http://deeptimekb.org/iscallnew>
 {
    {
       ?tconcept  a gts:GeochronologicEra ,
                 gts:Age;  
               skos:inScheme ', schemeList$schURI[k] , '; 
               rdfs:label ?lbl ;
               time:hasBeginning ?basePosition ;
               time:hasEnd ?topPosition. 
      ?basePosition time:inTemporalPosition ?baseTime .
      ?topPosition time:inTemporalPosition ?topTime .

      ?baseTime time:numericPosition ?baseNum .
      ?topTime time:numericPosition ?topNum .
    }
   UNION 
    { 
       ?tconcept  a gts:GeochronologicEra ,
                 gts:Age;  
               skos:inScheme ', schemeList$schURI[k] , '; 
               rdfs:label ?lbl ;
               dc:description
                   [ 
                       time:hasBeginning ?basePosition ;
                       time:hasEnd ?topPosition ;
                       skos:inScheme ', schemeList$schURI[k] , '
                   ] . 
       ?basePosition time:inTemporalPosition ?baseTime .
       ?topPosition time:inTemporalPosition ?topTime .

       ?baseTime   dc:description
                   [ 
                      time:numericPosition ?baseNum ;
                      skos:inScheme ', schemeList$schURI[k] , '
                   ] .
       ?topTime   dc:description
                   [ 
                      time:numericPosition ?topNum ;
                      skos:inScheme ', schemeList$schURI[k] , '
                   ] .
     } 

    FILTER (?baseNum >= 150)
    FILTER (?topNum <= 150) 
  }
}
ORDER BY ?baseNum
LIMIT 1
 '
     )
res0 = SPARQL(endpoint, q0)$results

#this step is important, if res0 is null, then there will be an error for the query in the below  
if(length(res0)==0) 
{
    cat("NULL","\n\n") 
}
    
else 
{
    # This query will find and list all the concepts at different levels  
    q = paste0(
                  sparql_prefix, 
                      '
    SELECT DISTINCT ?All ?cLabelEn
    WHERE
    {
    GRAPH <http://deeptimekb.org/iscallnew>
    {

    #for the local vocabularies
    {
        { #This block is to include the Age-level concept in the resulting list 
          ', res0$tconcept[1], ' rdfs:label ?labelAgeC . 
          ?All rdfs:label ?labelAgeC .
          FILTER (?All = ', res0$tconcept[1], ') #only use label is not enough to make sure the two concepts are the same
        }
        UNION
        { # This and the next block are to find the broader and broaderTransitive concepts
            ', res0$tconcept[1], ' skos:broaderTransitive ?All  
        }
        UNION 
        {
            ', res0$tconcept[1], ' skos:broader ?All  
        }

          ?All 
          rdfs:label ?cLabelEn ;
            time:hasBeginning ?baseboundary1 ;
            time:hasEnd ?topboundary1  .

          ?baseboundary1 time:inTemporalPosition ?basetime1 .
          ?topboundary1 time:inTemporalPosition ?toptime1 .

          ?basetime1 time:numericPosition ?baseNum1 .
          ?toptime1 time:numericPosition ?topNum1 .

    }

    UNION
    #for the local vocabularies
    {
        { #This block is to include the Age-level concept in the resulting list 
             ', res0$tconcept[1], ' rdfs:label ?labelAgeC . 
              ?All rdfs:label ?labelAgeC .
              FILTER (?All = ', res0$tconcept[1], ') #only use label is not enough to make sure the two concepts are the same
        }
        UNION
        { # This and the next block are to find the broader and broaderTransitive concepts
        ', res0$tconcept[1], ' 
        dc:description
              [
                 skos:broaderTransitive ?All ;
                 skos:inScheme  ', schemeList$schURI[k] , '
              ]
        }
        UNION 
        {
         ', res0$tconcept[1], ' 
        dc:description
              [
                 skos:broader  ?All ;
                 skos:inScheme ', schemeList$schURI[k] , '
              ]
        }

          ?All 
          rdfs:label ?cLabelEn ;
          dc:description 
          [  
            time:hasBeginning ?baseboundary1 ;
            time:hasEnd ?topboundary1 ;
            skos:inScheme ', schemeList$schURI[k] , ' 
            ] .

          ?baseboundary1 time:inTemporalPosition ?basetime1 .
          ?topboundary1 time:inTemporalPosition ?toptime1 .

          ?basetime1 dc:description
          [
            time:numericPosition ?baseNum1;
            skos:inScheme ', schemeList$schURI[k] , ' 
            ]  .

          ?toptime1 dc:description
          [
            time:numericPosition ?topNum1;
            skos:inScheme ', schemeList$schURI[k] , ' 
            ]  .
    }

        }
    }

    ORDER BY (?baseNum1 - ?topNum1)
                      '
                  )
    # run the query  
    res = SPARQL(endpoint, q)$results

    print(res)      
}

}



 ========[1] "Input age boundary number: 150"
[1] "<http://resource.geosciml.org/vocabulary/timescale/tsnc2019>"
[1] "Ontology for Geologic Timescale Elements in North China"
NULL 



 ========[1] "Input age boundary number: 150"
[1] "<http://resource.geosciml.org/vocabulary/timescale/tssc2019>"
[1] "Ontology for Geologic Timescale Elements in South China"
                                      All            cLabelEn
1 <http://deeptimekb.org/tssc#Suiningian> "Suiningian Age"@en


 ========[1] "Input age boundary number: 150"
[1] "<http://resource.geosciml.org/vocabulary/timescale/tsnz2019>"
[1] "Ontology for Geologic Timescale Elements in New Zealand"
                                   All         cLabelEn
1 <http://deeptimekb.org/tsnz#Puaroan> "Puaroan Age"@en
2   <http://deeptimekb.org/tsnz#Oteke> "Oteke Epoch"@en


 ========[1] "Input age boundary number: 150"
[1] "<http://resource.geosciml.org/vocabulary/timescale/tsau2019>"
[1] "Ontology for Geologic Timescale Elements in Austra

In [57]:
# This query will take almost two minutes 
# Below is another way to print out the lists of concepts 

ConceptLevels <- c("Age", "Epoch", "Sub-Period", "Period", "Era", "Eon", "Super-Eon")

cLevelLength = length(ConceptLevels)

# print out concepts (where  the input number locates in) in each vocabulary scheme 

#the looping by k is to loop through all vocabulary schemes
for (k in 1:schListLength) {
cat("\n\n","========")
print ('Input age boundary number: 150')
print(schemeList$schURI[k])
print(schemeList$schemeName[k])  

# In the sparql code below we use a UNION clause to tackel the ontology patterns for 
#  version control of the international vocabulary schemes 

#the looping by j is to loop through all concept levels from Age to Super-Eon
    
#we must filter both the base and top time - if only filter by base time, some error results will occur    
for (j in 1:cLevelLength)  {  
q = paste0(
              sparql_prefix, 
                  '
SELECT DISTINCT ?tconcept  str(?lbl) AS ?cLabel
WHERE
{
    {
       ?tconcept  a gts:GeochronologicEra ,
                 gts:', ConceptLevels[j] , ';  
               skos:inScheme ', schemeList$schURI[k] , '; 
               rdfs:label ?lbl ;
               time:hasBeginning ?basePosition ;
               time:hasEnd ?topPosition. 
      ?basePosition time:inTemporalPosition ?baseTime .
      ?topPosition time:inTemporalPosition ?topTime .

      ?baseTime time:numericPosition ?baseNum .
      ?topTime time:numericPosition ?topNum .
    }
   UNION 
    { 
       ?tconcept  a gts:GeochronologicEra ,
                 gts:', ConceptLevels[j] , ';  
               skos:inScheme ', schemeList$schURI[k] , '; 
               rdfs:label ?lbl ;
               dc:description
                   [ 
                       time:hasBeginning ?basePosition ;
                       time:hasEnd ?topPosition ;
                       skos:inScheme ', schemeList$schURI[k] , '
                   ] . 
       ?basePosition time:inTemporalPosition ?baseTime .
       ?topPosition time:inTemporalPosition ?topTime .

       ?baseTime   dc:description
                   [ 
                      time:numericPosition ?baseNum ;
                      skos:inScheme ', schemeList$schURI[k] , '
                   ] .
       ?topTime   dc:description
                   [ 
                      time:numericPosition ?topNum ;
                      skos:inScheme ', schemeList$schURI[k] , '
                   ] .
     } 

    FILTER (?baseNum >= 150)
    FILTER (?topNum <= 150) 

}
ORDER BY ?baseNum
LIMIT 1
        '
          )
# run the query  
resConcepts = SPARQL(endpoint, q)$results

cat(ConceptLevels[j], "Level: ", resConcepts$cLabel[1], "\n")
    
}
}    



 ========[1] "Input age boundary number: 150"
[1] "<http://resource.geosciml.org/vocabulary/timescale/tsnc2019>"
[1] "Ontology for Geologic Timescale Elements in North China"
Age Level:  
Epoch Level:  
Sub-Period Level:  
Period Level:  
Era Level:  
Eon Level:  
Super-Eon Level:  


 ========[1] "Input age boundary number: 150"
[1] "<http://resource.geosciml.org/vocabulary/timescale/tssc2019>"
[1] "Ontology for Geologic Timescale Elements in South China"
Age Level:  Suiningian Age 
Epoch Level:  
Sub-Period Level:  
Period Level:  
Era Level:  
Eon Level:  
Super-Eon Level:  


 ========[1] "Input age boundary number: 150"
[1] "<http://resource.geosciml.org/vocabulary/timescale/tsnz2019>"
[1] "Ontology for Geologic Timescale Elements in New Zealand"
Age Level:  Puaroan Age 
Epoch Level:  Oteke Epoch 
Sub-Period Level:  
Period Level:  
Era Level:  
Eon Level:  
Super-Eon Level:  


 ========[1] "Input age boundary number: 150"
[1] "<http://resource.geosciml.org/vocabulary/timescale

<0 x 0 matrix>